In [1]:
import numpy as np
import pandas as pd
import openpyxl as op
import datetime
%run preprocessing_methods.ipynb

In [2]:
# reading the data
path = "AAV_MPA_GPA_v2.xlsx"
wb_obj = op.load_workbook(path, data_only=True)
sheet_obj = wb_obj.active
max_row = sheet_obj.max_row
data = pd.read_excel(path, nrows=max_row - 5)
data.columns = get_header(path, max_row)

data.shape
data

,"Data stworzenia, ,","Kod pacjenta, ,","Ogólne, Płeć, K","Ogólne, Płeć, M","Ogólne, Data urodzenia,","Ogólne, Rozpoznanie kliniczne, Zapalenie tętnic Takayasu","Ogólne, Rozpoznanie kliniczne, Olbrzymiokomórkowe zapalenie tętnic","Ogólne, Rozpoznanie kliniczne, Guzkowe zapalenie tętnic","Ogólne, Rozpoznanie kliniczne, Choroba Kawasakiego","Ogólne, Rozpoznanie kliniczne, Mikroskopowe zapalenie naczyń (MPA)",...,"Powikłania leczenia immunosupresyjnego , Narząd wzroku/Ucho/Nos/Gardło , Brak danych","Powikłania leczenia immunosupresyjnego , Serce/Płuca, Tak","Powikłania leczenia immunosupresyjnego , Serce/Płuca, Nie","Powikłania leczenia immunosupresyjnego , Serce/Płuca, Brak danych","Powikłania leczenia immunosupresyjnego , Nerki, Tak","Powikłania leczenia immunosupresyjnego , Nerki, Nie","Powikłania leczenia immunosupresyjnego , Nerki, Brak danych","Powikłania leczenia immunosupresyjnego , Objawy ogólne , Tak","Powikłania leczenia immunosupresyjnego , Objawy ogólne , Nie","Powikłania leczenia immunosupresyjnego , Objawy ogólne , Brak danych"
0,2016/08/14,WIRMO0025,1,0,1991/12/30,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,2017/06/24,KRKWS0004,1,0,1953/12/13,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,2016/06/03,PUMMB0066,1,0,1978/07/05,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,2016/11/04,SUMEK0002,1,0,1972/05/02,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,2016/06/04,SUMJD0007,0,1,1960/08/01,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,2016/07/28,SUMJD0009,1,0,1959/02/01,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
884,2020/04/10,GUMZZ0259,1,0,1959/01/09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
885,2016/03/20,KRKJG0007,0,1,1966/08/03,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886,2019/09/18,GUMZZ0148,0,1,1943/10/01,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data.drop_duplicates(subset=["Kod pacjenta, , "], inplace=True)
data.shape

(719, 260)

In [4]:
# new columns

data["Data stworzenia, , "] = pd.to_datetime(data["Data stworzenia, , "])
data["Ogólne, Data urodzenia, "] = pd.to_datetime(data["Ogólne, Data urodzenia, "])
data["Ogólne, Data rozpoznania, "] = pd.to_datetime(data["Ogólne, Data rozpoznania, "])
data["Ogólne, Data wystąpienia pierwszych objawów, "] = pd.to_datetime(data["Ogólne, Data wystąpienia pierwszych objawów, "])

# reparing data
data["Ogólne, Data wystąpienia pierwszych objawów, "] = np.where(data["Ogólne, Data wystąpienia pierwszych objawów, "].isnull(), data["Ogólne, Data rozpoznania, "], data["Ogólne, Data wystąpienia pierwszych objawów, "])
data["Ogólne, Data rozpoznania, "] = np.where(data["Ogólne, Data rozpoznania, "].isnull(), data["Ogólne, Data wystąpienia pierwszych objawów, "], data["Ogólne, Data rozpoznania, "])

data['wiek zachorowania'] = data["Ogólne, Data rozpoznania, "] - data["Ogólne, Data urodzenia, "]
data['opóźnienie rozpoznania'] = data["Ogólne, Data rozpoznania, "] - data["Ogólne, Data wystąpienia pierwszych objawów, "]
data['czas trwania choroby'] = data["Data stworzenia, , "] - data["Ogólne, Data rozpoznania, "]

mask = data['wiek zachorowania'].notna()
data = data[mask]
   
data['wiek zachorowania'] = (data['wiek zachorowania'].dt.days/30).round()
data['opóźnienie rozpoznania'] = (data['opóźnienie rozpoznania'].dt.days/30).round()
data['czas trwania choroby'] = (data['czas trwania choroby'].dt.days/30).round()
# data.to_excel("1.xlsx")
data.shape

c:\python\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\python\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\python\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

(716, 263)

In [5]:
df = data.copy()

# biopsies
df.drop(df.columns[204:227], axis=1, inplace=True)

# course of illness
df.drop("Leczenie, Inne , ", axis=1, inplace=True)
df.drop(df.columns[105:114], axis=1, inplace=True)
df.drop("Manifestacja kliniczna zapalenia naczyń, Inne objawy, ", axis=1, inplace=True)

# general
df.drop(df.columns[48:60], axis=1, inplace=True)
df.drop(df.columns[39:47], axis=1, inplace=True)
df.drop("Ogólne, Inne, ", axis=1, inplace=True)

# others diseases
df.drop(df.columns[11:37], axis=1, inplace=True)
df.drop(df.columns[5:9], axis=1, inplace=True)

# date
df.drop("Data stworzenia, , ", axis=1, inplace=True)
df.drop("Ogólne, Data urodzenia, ", axis=1, inplace=True)
df.drop("Ogólne, Data rozpoznania, ", axis=1, inplace=True)
df.drop("Ogólne, Data wystąpienia pierwszych objawów, ", axis=1, inplace=True)
df.drop("Przebieg choroby, Zgon pacjenta, ", axis=1, inplace=True)

# df.to_excel("2.xlsx")
df.shape

(716, 171)

In [6]:
one_unit_creatinine(df)
df.drop("Przebieg choroby, Maksymalne stężenie kreatyniny, umol/l", axis=1, inplace=True)
df.drop("Przebieg choroby, Maksymalne stężenie kreatyniny, mg/dl", axis=1, inplace=True)
df.drop("Przebieg choroby, Maksymalne stężenie kreatyniny, ", axis=1, inplace=True)

# df.to_excel("3.xlsx")
df.shape

(716, 169)

In [7]:
df.drop("Kod pacjenta, , ", axis=1, inplace=True)
df.drop(get_list_of_deaths_columns(), axis=1, inplace=True)
df.drop(get_list_of_dialysis_columns(), axis=1, inplace=True)

df.shape

(716, 158)

In [8]:
replace_value(df, np.nan, 0)
replace_value(df, "Nie dotyczy", 0)
replace_value(df, "nie dotyczy", 0)
replace_value(df, "-", 0)
replace_value(df, "", 0)

df.to_excel("4.xlsx")
df.shape

(716, 158)

In [9]:
# dropping patiens without GPA or MPA

df = df[(df["Ogólne, Rozpoznanie kliniczne, Mikroskopowe zapalenie naczyń (MPA)"] == 1) | 
       (df["Ogólne, Rozpoznanie kliniczne, Ziarniakowatość z zapaleniem naczyń (Wegenera)(GPA)"]==1)]

In [10]:
df = df[df['wiek zachorowania'].notna()]
df = df[df['opóźnienie rozpoznania'].notna()]
df = df[df['czas trwania choroby'].notna()]

df = df[df["wiek zachorowania"] > 0]
df[df["opóźnienie rozpoznania"] < 0] = 0
df = df[df["czas trwania choroby"] > 0]

# df.to_excel("5.xlsx")
df.shape

(684, 158)

In [11]:
mpa_ids = df["Ogólne, Rozpoznanie kliniczne, Mikroskopowe zapalenie naczyń (MPA)"] 
gpa_ids = df["Ogólne, Rozpoznanie kliniczne, Ziarniakowatość z zapaleniem naczyń (Wegenera)(GPA)"]

In [15]:
df

,"Ogólne, Płeć, K","Ogólne, Płeć, M","Ogólne, Rozpoznanie kliniczne, Mikroskopowe zapalenie naczyń (MPA)","Ogólne, Rozpoznanie kliniczne, Ziarniakowatość z zapaleniem naczyń (Wegenera)(GPA)","Manifestacja kliniczna zapalenia naczyń, Objawy ogólne , Tak","Manifestacja kliniczna zapalenia naczyń, Objawy ogólne , Nie","Manifestacja kliniczna zapalenia naczyń, Objawy ogólne , Brak danych","Manifestacja kliniczna zapalenia naczyń, Układ mięśniowo-szkieletowy , Tak","Manifestacja kliniczna zapalenia naczyń, Układ mięśniowo-szkieletowy , Nie","Manifestacja kliniczna zapalenia naczyń, Układ mięśniowo-szkieletowy , Brak danych",...,"Powikłania leczenia immunosupresyjnego , Serce/Płuca, Brak danych","Powikłania leczenia immunosupresyjnego , Nerki, Tak","Powikłania leczenia immunosupresyjnego , Nerki, Nie","Powikłania leczenia immunosupresyjnego , Nerki, Brak danych","Powikłania leczenia immunosupresyjnego , Objawy ogólne , Tak","Powikłania leczenia immunosupresyjnego , Objawy ogólne , Nie","Powikłania leczenia immunosupresyjnego , Objawy ogólne , Brak danych",wiek zachorowania,opóźnienie rozpoznania,czas trwania choroby
0,1,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,192.0,61.0,108.0
1,1,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,745.0,4.0,28.0
2,1,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,333.0,7.0,128.0
3,1,0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,476.0,1.0,66.0
4,0,1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,656.0,2.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,1,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,577.0,0.0,75.0
879,1,0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,255.0,16.0,1.0
881,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,452.0,0.0,269.0
883,1,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,620.0,0.0,80.0
